# Amplitude Amplification Algorithm

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
%load_ext qat.core.magic

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()    

## 1. Data Discretization

In [ ]:
from AuxiliarFunctions import  get_histogram, PostProcessResults, TestBins, RunJob
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 15
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

## 2. Circuit Implementation

In [ ]:
from QuantumMultiplexors_Module_examples import ExpectationLoadingData
from PhaseAmplification_Module import Load_Q_Gate

In [ ]:
K = 2
#Initial State
Qprog, P_Gate, R_gate = ExpectationLoadingData(p_X, f_X)
qbits = Qprog.registers
Q_Gate = Load_Q_Gate(P_Gate, R_gate)
for k in range(K):
    print(k)
    Qprog.apply(Q_Gate, qbits)

circuit = Qprog.to_circ(submatrices_only=True)
job = circuit.to_job(qubits=[n_qbits])

In [ ]:
%qatdisplay circuit

In [ ]:
result = RunJob(lineal_qpu.submit(job))
Phi = PostProcessResults(result)

In [ ]:
Phi

Como aprovechar esto: 
A priori la forma es bastante directa: Aplicamos el operador K veces por lo que:

* Estado despues de la Carga de Datos: $|\Psi\rangle=\sin{\theta}|\Psi_{1}\rangle+\cos{\theta}|\Psi_{0}\rangle$
* Estado al final del circuito: $\hat{Q}^{K}|\Psi\rangle = \sin{((2*K+1)\theta)}|\Psi_{1}\rangle+\cos{((2*K+1)\theta})|\Psi_{0}\rangle$

Realizamos N medidas del último qbit y generamos las probabilidades asociadas a medir $|\Psi_{1}\rangle$ y $|\Psi_{1}\rangle$. Esto generaría los siguientes números:

* $P_{|\Psi_{1}\rangle}$: Probabilidad de haber medido el estado $|\Psi_{1}\rangle$ que esperamos sea muy alta
* $P_{|\Psi_{0}\rangle}$: Probabilidad de haber medido el estado $|\Psi_{0}\rangle$ que esperamos sea muy baja

A continuación igualamos las probabilidades con las amplitudes:

$$\sin^{2}((2*K+1)\theta) = P_{|\Psi_{1}\rangle}$$
$$\sin((2*K+1)\theta) = \sqrt{P_{|\Psi_{1}\rangle}}$$
$$(2*K+1)\theta = \arcsin{\sqrt{P_{|\Psi_{1}\rangle}}}$$

Y aquí paramos!!! porque al hacer el $\arcsin$ esto es una función multivaluada pero la función numpy solo calcula el ángulo entre $[\frac{-\pi}{2}, \frac{\pi}{2}]$. Es decir si el operador $\hat{Q}^K$ hace rotaciones que dejen el vector en un ángulo superior a $\frac{\pi}{2}$ el ángulo que nos devolverá la función no será el que buscamos: $(2*K+1)\theta$ sino el ángulo que tenga el mismo valor del $\sin$ pero en el intervalo anterior.. por lo que no podemos hacer la divsión para obtener el $\theta$ original ...

Veamos primero con lo que tenemos que esto NO funciona: en la siguiente celda calculamos $(2*K+1)\theta$ utilizando la probabilidad de obtener el estado: $P_{|\Psi_{1}\rangle}$

In [ ]:
theta_K = np.arcsin((Phi['Probability'].iloc[1])**0.5)
print('theta_K: {}'.format(theta_K))

Ahora obtenemos el ángulo $\theta$ correspondiente a: $\theta = \frac{\arcsin{\sqrt{P_{|\Psi_{1}\rangle}}}}{(2*K+1)}$$


In [ ]:
thetaFromK = theta_K/(2*K+1)
print('thetaFromK: {}'.format(thetaFromK))

No obstante sabemos que:

$$\sin \theta = \sqrt{\sum_{x=0}^{2^-1} p(x)f(x)}$$

$$ \theta = \arcsin{\sqrt{\sum_{x=0}^{2^-1} p(x)f(x)}}$$

Esto lo calculamos en la siguiente celda!!!

In [ ]:
theta_teorico = np.arcsin(np.sqrt(sum(p_X*f_X)))
print('theta_teorico: {}'.format(theta_teorico))

Como podemos observar **thetaFromK** es distinto de **theta_teorico** ($\theta_{th}$). No obstante si ahora hacemos:

$$\sin^2((2*K+1)\theta$$

Obtendremos $P_{|\Psi_{1}\rangle}$

In [ ]:
print('Probabilidad de |Psi_1> a partir de theta_teorico: {}'.format(np.sin((2*K+1)*theta_teorico)**2))

print(np.isclose(np.sin((2*K+1)*theta_teorico)**2, Phi['Probability'].iloc[1]))

Aquí vemos el problema. De momento NO tenemos nada ya que no podemos extraer el valor de $(2*K+1)\theta$ correcto!!!